In [1]:
import tensorflow as tf
from keras.api.preprocessing import image_dataset_from_directory

# Paths for training, testing, and validation images
train_path = 'nude_sexy_safe_v1_x320/training'
test_path = 'nude_sexy_safe_v1_x320/testing'
validation_path = 'nude_sexy_safe_v1_x320/validation'

# Number of classes (update this based on your dataset)
num_classes = 3

# Preprocessing pipeline for training data
preprocessing_pipeline_train = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
    tf.keras.layers.Resizing(height=64, width=64),
    tf.keras.layers.RandomCrop(height=64, width=64),
    tf.keras.layers.RandomRotation(factor=0.2, fill_mode="reflect", interpolation="bilinear"),
    tf.keras.layers.RandomZoom(height_factor=(0.2, 0.3), fill_mode="reflect", interpolation="bilinear"),
    tf.keras.layers.RandomBrightness(factor=0.1)
])

# Preprocessing pipeline for testing and validation data
preprocessing_pipeline_test = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
])

# Create a subset of the train dataset for demo purposes (e.g., limit to 1000 images)
train_dataset = image_dataset_from_directory(
    train_path,
    label_mode='int',
    batch_size=32,
    image_size=(64, 64),
    shuffle=True,     # Shuffle to get a random subset
    seed=42           # Set seed for reproducibility
)

# Limit the number of batches to a smaller subset (for example, 50 batches * 32 images = 1600 images)
train_dataset = train_dataset.take(50)
train_dataset = train_dataset.map(lambda x, y: (preprocessing_pipeline_train(x), tf.one_hot(y, num_classes)))

# Create the validation dataset from the 'validation' directory
validation_dataset = image_dataset_from_directory(
    validation_path,
    label_mode='int',
    batch_size=32,
    image_size=(64, 64)
)
validation_dataset = validation_dataset.map(lambda x, y: (preprocessing_pipeline_test(x), tf.one_hot(y, num_classes)))

# Create test dataset
test_dataset = image_dataset_from_directory(
    test_path,
    label_mode='int',
    batch_size=32,
    image_size=(64, 64)
)
test_dataset = test_dataset.map(lambda x, y: (preprocessing_pipeline_test(x), tf.one_hot(y, num_classes)))

# CNN model
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.Input(shape=(64, 64, 3)))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the subset of training data and validate with the validation dataset
cnn.fit(
    train_dataset,
    validation_data=validation_dataset,  # Use the separate validation dataset
    epochs=5
)

# Evaluate the model on the test dataset
test_loss, test_acc = cnn.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")


2024-09-11 14:28:21.706201: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 14:28:21.909278: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-11 14:28:22.702071: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 14:28:24.820422: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 664792 files belonging to 3 classes.
Found 10306 files belonging to 3 classes.
Found 10049 files belonging to 3 classes.
Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 376ms/step - accuracy: 0.5693 - loss: 2.1921 - val_accuracy: 0.3881 - val_loss: 1.0868
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 364ms/step - accuracy: 0.6326 - loss: 0.8451 - val_accuracy: 0.3881 - val_loss: 1.0923
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 377ms/step - accuracy: 0.6385 - loss: 0.8658 - val_accuracy: 0.3881 - val_loss: 1.1004
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 357ms/step - accuracy: 0.6530 - loss: 0.8150 - val_accuracy: 0.3881 - val_loss: 1.0966
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 391ms/step - accuracy: 0.6395 - loss: 0.8363 - val_accuracy: 0.3881 - val_loss: 1.0933
315/315 ━━━━━━━━━━━━━━━━━━━━ 17s 54ms/step - accuracy: 0.3891 - loss: 1.0769
Test Accuracy: 0.3859
